In [22]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
#install dependencies
!pip install python_speech-features
!apt install ffmpeg
!pip install wavio
!pip install pydub

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [24]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
import subprocess
from tqdm import tqdm
from scipy.io import wavfile
import wavio
import librosa
from python_speech_features import mfcc, logfbank
from pydub import AudioSegment
from pydub.utils import mediainfo
 
#display max row:500
pd.set_option('display.max_rows', 500)
#change directory to base folder
os.chdir('/content/drive/My Drive/GLP')

In [ ]:
#unzip audio files
!unzip -q 'audio_files.zip'
!unzip -q 'AdditionalUtterances.zip'
!unzip -q 'nlp_keywords_29Oct2020.zip'

In [8]:
#check files in directory
!ls

AdditionalUtterances.zip  GLP1.py	      nlp_keywords
audio_files		  GLP_data_PPP.ipynb  nlp_keywords_29Oct2020.zip
audio_files.zip		  keywords	      SampleSubmission.csv
clean.py		  latest_keywords     Train.csv


In [26]:
#read csv files
train = pd.read_csv('Train.csv')
 
submission = pd.read_csv('SampleSubmission.csv')
 
train['fn'] = train['fn'].str[12:]
submission['fn'] = submission['fn'].str[12:]

In [25]:
#print the number of files and folders 
audio_files_no = sum(len(files) for _, _, files in os.walk('audio_files'))
latest_keywords_no = sum(len(files) for _, _, files in os.walk('latest_keywords'))
latest_keywords_fl = sum(len(folders) for _, folders, _ in os.walk('latest_keywords'))
nlp_keywords_no = sum(len(files) for _, _, files in os.walk('nlp_keywords'))
nlp_keywords_fl = sum(len(folders) for _, folders, _ in os.walk('nlp_keywords'))

print('audio_files: '+ str(audio_files_no), 
      'labels: '+ str(len(train['label'].unique())),
      'latest_keywords: '+ str(latest_keywords_no), 
      'latest_keywords_folders: '+ str(latest_keywords_fl), 
      'nlp_keywords: '+ str(nlp_keywords_no), 
      'nlp_keywords_folders: '+str(nlp_keywords_fl))

audio_files: 2126 labels: 193 latest_keywords: 1740 latest_keywords_folders: 129 nlp_keywords: 1861 nlp_keywords_folders: 122


In [9]:
if len(os.listdir('keywords')) == 0:
    for f in tqdm(train['label'].unique()):
        dir = f
        p_dir = 'keywords'
        path = os.path.join(p_dir, dir)
        os.mkdir(path)

100%|██████████| 193/193 [00:00<00:00, 453.49it/s]


In [10]:
for i in tqdm(range(len(train))):
    f_dir1 = 'audio_files'
    f_dir2 = str(train.loc[i, 'fn'])
    d_dir1 = 'keywords'
    d_dir2 = str(train.loc[i, 'label'])
    f_dir = os.path.join(f_dir1, f_dir2)
    d_dir = os.path.join(d_dir1, d_dir2)
    shutil.copy2(f_dir, d_dir)

100%|██████████| 1109/1109 [08:28<00:00,  2.18it/s]


In [11]:
for i in tqdm(range(len(submission))):
    f_dir1 = '/content/drive/My Drive/GLP/audio_files'
    f_dir2 = str(submission.loc[i, 'fn'])
    f_dir = os.path.join(f_dir1, f_dir2)
    d_dir = '/content/drive/My Drive/GLP/submission'
    shutil.copy2(f_dir, d_dir)

100%|██████████| 1017/1017 [05:10<00:00,  3.27it/s]


In [29]:
if len(os.listdir('raw_train')) == 0:
    for f in tqdm(train['label'].unique()):
        dir = f
        p_dir = 'raw_train'
        path = os.path.join(p_dir, dir)
        os.mkdir(path)

100%|██████████| 193/193 [00:00<00:00, 464.71it/s]


In [38]:
def convert_webm_to_wav(f,old_dir,new_dir):
    #ffmpeg -i ./big-buck-bunny_trailer.webm -c:a pcm_f32le ./out.wav
    command = ['ffmpeg', '-i', os.path.join(old_dir,f), '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '22050', os.path.join(new_dir,f)]
    subprocess.run(command,stdout=subprocess.PIPE,stdin=subprocess.PIPE)

lk_path = '/content/drive/My Drive/GLP/latest_keywords'
for dirpath, _, filenames in os.walk(lk_path):
  for filename in filenames:
    f = filename
    od_dir = dirpath
    s = '/content/drive/My Drive/GLP/raw_train'
    n = str(dirpath[43:])
    nw_dir = s + n
    convert_webm_to_wav(f, od_dir, nw_dir)

In [39]:
k_path = '/content/drive/MyDrive/GLP/keywords'
for dirpath, _, filenames in os.walk(k_path):
  for filename in filenames:
    f = filename
    od_dir1 = dirpath
    s = '/content/drive/My Drive/GLP/raw_train'
    n = str(dirpath[35:])
    nw_dir1 = s + n
    convert_webm_to_wav(f, od_dir1, nw_dir1)

In [40]:
nk_path = '/content/drive/My Drive/GLP/nlp_keywords'
for dirpath, _, filenames in os.walk(nk_path):
  for filename in filenames:
    f = filename
    od1_dir = dirpath
    s = '/content/drive/My Drive/GLP/raw_train'
    n = str(dirpath[40:])
    nw1_dir = s + n
    convert_webm_to_wav(f, od1_dir, nw1_dir)

In [36]:
s_path = '/content/drive/My Drive/GLP/submission'
for dirpath, _, filenames in os.walk(s_path):
  for filename in filenames:
    f = filename
    od2_dir = dirpath
    s = '/content/drive/My Drive/GLP/raw_test'
    nw2_dir = s 
    convert_webm_to_wav(f, od2_dir, nw2_dir)

bmission


In [ ]:
#def CopyFol_SFolders(src, dst):
#    for i in os.listdir(src):
#        s_dir = os.path.join(src, i)
#        d_dir = os.path.join(dst, i)
#        if os.path.isdir(s_dir):
#            CopyFol_SFolders(s_dir, d_dir)
#        else:
#            shutil.copy2(s_dir, d_dir)

In [ ]:
#CopyFol_SFolders('latest_keyword', 'keywords')

In [43]:
print('raw_train: '+ str(sum(len(files) for _, _, files in os.walk('raw_train'))))
print('raw_test: '+ str(sum(len(files) for _, _, files in os.walk('raw_test'))))

raw_train: 4709
raw_test: 1017


In [44]:
print('keywords: '+ str(sum(len(files) for _, _, files in os.walk('keywords'))))
print('nlp_keywords: '+ str(sum(len(files) for _, _, files in os.walk('nlp_keywords'))))
print('latest_keywords: '+ str(sum(len(files) for _, _, files in os.walk('latest_keywords'))))

keywords: 1109
nlp_keywords: 1861
latest_keywords: 1740
